<a href="https://colab.research.google.com/github/JiseokSeo/Toy_Projects/blob/main/selenium_3_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install selenium
# !pip install chromedriver-autoinstaller

In [3]:
from selenium import webdriver
import chromedriver_autoinstaller
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import re

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
chrome_ver

In [ ]:
try:
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
except:
    crawler = chromedriver_autoinstaller.install(True)
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
crawler.implicitly_wait(10)

# 네이버쇼핑에서 중분류 카테고리까지 들어가셔서 링크를 넣어주세요요
crawling_url = 'https://search.shopping.naver.com/search/category/100007966?catId=50012620%2050013360&origQuery&pagingIndex=1&pagingSize=40&productSet=total&query&sort=rel&timestamp=&viewType=list'
crawler.get(crawling_url)

In [ ]:
cat1 = crawler.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[1]').text
cat2 = crawler.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[2]').text
cat3 = crawler.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[3]').text
cat1, cat2, cat3 # 각 카테고리명입니다 

('식품', '오일/가루/향신료', '4')

In [ ]:
cat_list = [f'//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[{i}]/a/span' for i in range(1, int(cat3)+1)]
# 방문해야하는 소분류 리스트입니다 

In [ ]:
data = [] # 최종 크롤링 산물이 들어갈 리스트트

In [ ]:
for cat in cat_list:
    crawler.get(crawling_url)
    time.sleep(2)
    
    crawler.find_element('xpath', cat).click()
    cat_now = crawler.find_element('xpath', cat).text
    time.sleep(2)
    
    url = crawler.current_url
    cat_code = re.findall('/[0-9]{9}', url)[0][1:]
    cat_id = re.findall('=[0-9]{8}', url)[0][1:]

    for i in range(1, 145):  # 탐색할 인덱스 번호 구간 
        crawler.get(f'https://search.shopping.naver.com/search/category/{cat_code}?catId={cat_id}&origQuery&pagingIndex={i}&pagingSize=20&productSet=total&query&sort=rel&timestamp=&viewType=list')
        print(f'now {cat_now}:{i} page')
        
        while True:
            time.sleep(1)
            bh = crawler.execute_script('return document.body.scrollHeight')

            crawler.execute_script('window.scrollTo(0, document.body.scrollHeight)')

            ah = crawler.execute_script('return document.body.scrollHeight')

            if ah == bh:
                break
                time.sleep(1)
                
            bh = ah
            crawler.execute_script('window.scrollTo(document.body.scrollHeight, 0)')
        
        time.sleep(3)
        infos = crawler.find_elements(By.CSS_SELECTOR, '.basicList_info_area__TWvzp')
        for info in infos:
            try:
                name = info.find_element(By.CSS_SELECTOR, '.basicList_title__VfX3c > a').text
                price = info.find_element(By.CSS_SELECTOR, 'div.basicList_info_area__TWvzp > div.basicList_price_area__K7DDT > strong > span > span.price_num__S2p_v').text
                data.append([name, cat2, cat_now, price])
    #             if len(data) >= 3000:
    #                 break
            except:
                pass
        
        print(f'size of df: {len(data)}\n')
#         if len(data) >= 3000:
#             break

now 향신료:1 page
size of df: 6972

now 향신료:2 page
size of df: 6998

now 향신료:3 page
size of df: 7024

now 향신료:4 page
size of df: 7050

now 향신료:5 page
size of df: 7076

now 향신료:6 page
size of df: 7102

now 향신료:7 page
size of df: 7128

now 향신료:8 page
size of df: 7154

now 향신료:9 page
size of df: 7180

now 향신료:10 page
size of df: 7206

now 향신료:11 page
size of df: 7232

now 향신료:12 page
size of df: 7258

now 향신료:13 page
size of df: 7284

now 향신료:14 page
size of df: 7310

now 향신료:15 page
size of df: 7336

now 향신료:16 page
size of df: 7362

now 향신료:17 page
size of df: 7386

now 향신료:18 page
size of df: 7406

now 향신료:19 page
size of df: 7426

now 향신료:20 page
size of df: 7446

now 향신료:21 page
size of df: 7466

now 향신료:22 page
size of df: 7486

now 향신료:23 page
size of df: 7506

now 향신료:24 page
size of df: 7526

now 향신료:25 page
size of df: 7546

now 향신료:26 page
size of df: 7566

now 향신료:27 page
size of df: 7586

now 향신료:28 page
size of df: 7606

now 향신료:29 page
size of df: 7626

now 향신료:30 page
size of

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(columns=['title', 'cat1', 'cat2', 'price'], data=data)

In [ ]:
df.to_csv(f'오일_가루_향신료.csv')